# Traitement des informations sur la ligne de temps

Importation des modules

- pandas pour la gestion des csv et les calculs type excel
- numpy pour les calculs en général
- matplotlib pour les graphiques

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy, datetime
import io

# actions per time interval:
timeinterval=5.0
before=timeinterval/2 # time before in secs
after=timeinterval/2  # time after in secs 

# plages de rythme :
rythme1=3
rythme2=6

In [2]:
def very_deep_copy(self):
    return pd.DataFrame(self.values.copy(), self.index.copy(), self.columns.copy())
pd.DataFrame.very_deep_copy = very_deep_copy

###Lecture des fichiers csv dans une DataFrame

- *Match* pour le fichier issu du traitement Rythme sur les actions et correspondances
- *Syntaxe* pour le fichier issu du traitement syntaxique de la transcription


In [3]:
Match=pd.read_csv("/Users/gilles/Downloads/CorrespondancesEvActionsCommALL-GB-Rythme.csv",sep=";")

In [4]:
Syntaxe=pd.read_csv("/Users/gilles/Downloads/RugbyFrance_Argentine_Annot_Revised-140701-GB.csv",sep=",")

###Mise en forme des champs de Match

- Conversion des champs Début et Fin en datetime
- Nettoyage des champs textes pour supprimer les blancs en trop
- Mise en place de l'index temporel

In [5]:
Match['Début']=pd.to_datetime(Match['Début'])
Match['Fin']=pd.to_datetime(Match['Fin'])

Match['NrAction']=Match['NrAction'].str.strip()
Match['Action']=Match['Action'].str.strip()
Match['Evénement']=Match['Evénement'].str.strip()
Match['RefCommentaire']=Match['RefCommentaire'].str.strip()
Match['Commentaire']=Match['Commentaire'].str.strip()
Match["Structure"]=Match["Structure"].str.lower().str.strip()
Match["Structure1"]=Match["Structure1"].str.lower().str.strip()
Match["Structure2"]=Match["Structure2"].str.lower().str.strip()
Match["Structure3"]=Match["Structure3"].str.lower().str.strip()

Match=Match.set_index(pd.DatetimeIndex(Match['Début']))

###Mise en forme des champs de Syntaxe

- Nettoyage des champs textes
- Elimination des lignes vides

In [6]:
Syntaxe['Commentaire']=Syntaxe['Commentaire'].str.strip()
Syntaxe=Syntaxe.dropna()

###Faire la liste des Commentaires dans Syntaxe
La liste permettra de rapprocher les données de Syntaxe de celles de Match via la colonne Commentaire

In [7]:
refsSyn=list(Syntaxe.Commentaire.values)

###Interpoloation de RythmeAction et ajout de Speaker et RépartitionSPK

- interpolate(method='time') fonctionne sur l'échelle de l'index (identique à Début)
- str.split() fonctionne comme un split() mais sur une colonne, on obtient les éléments avec str.get()
- str.lower() comme lower() sur une colonne


In [8]:
#Match['Durée']=Match['Fin']-Match['Début']
Match['RythmeAction']=Match['RythmeAction'].interpolate(method='time')
Match['Speaker']=Match['Commentaire'].str.split("-",1).str.get(0).str.lower().str.strip()

def repartitionSPK(row):
    duree=(row['Fin']-row['Début']).total_seconds()
    if duree>before+after:
        if row['Speaker']=="spk1":
            return "1;0;0;0"
        elif row['Speaker']=="spk2":
            return "0;1;0;0"
        elif row['Speaker']=="spk3":
            return "0;0;1;0"
        elif row['Speaker']=="":
            return "0;0;0;1"
        else:
            return "0;0;0;0"
    else:
        addBefore=before-duree/2
        addAfter=after-duree/2
        Extrait=Match.loc[(Match["Début"]>=row["Début"]-datetime.timedelta(seconds=int(addBefore)))&(Match["Fin"]<=row["Fin"]+datetime.timedelta(seconds=int(addAfter)))]
        nbSpk1=Extrait.loc[Extrait["Speaker"]=="spk1"]["Speaker"].count()
        nbSpk2=Extrait.loc[Extrait["Speaker"]=="spk2"]["Speaker"].count()
        nbSpk3=Extrait.loc[Extrait["Speaker"]=="spk3"]["Speaker"].count()
        nbNoComment=Extrait.loc[Extrait["Speaker"].isnull()]["Speaker"].shape[0]
        return "%d;%d;%d;%d" %(nbSpk1,nbSpk2,nbSpk3,nbNoComment)
Match['RépartitionSPK']=Match.apply(repartitionSPK, axis=1)

###Décomposition de la répartition en 4 colonnes

In [9]:
Match["RépartitionSpk1"]=Match['RépartitionSPK'].str.split(";").str.get(0).astype(int)
Match["RépartitionSpk2"]=Match['RépartitionSPK'].str.split(";").str.get(1).astype(int)
Match["RépartitionSpk3"]=Match['RépartitionSPK'].str.split(";").str.get(2).astype(int)
Match["RépartitionNoComm"]=Match['RépartitionSPK'].str.split(";").str.get(3).astype(int)

###Séparation des données en deux tables

- Actions pour les calculs sur les actions
- Commentaires pour les calculs sur les commentaires

In [10]:
Actions=Match[Match.Action.notnull()]
Commentaires=Match[Match.Commentaire.notnull()]

###Mise en forme de Actions

- Ajout de colonnes 
 - NrEvénement 
 - Evénement
 - NrAction 
 - SpkCommentaire : référence du commentaire de l'action
- Remplissage des vides de SpkCommentaire avec "SansCommentaire"

In [11]:
Actions=Actions.drop('Commentaire',1)
Actions=Actions.drop('ActionsAvant',1)
Actions=Actions.drop('ActionsAprès',1)
Actions['NrEvénement']=Actions['Evénement'].str.split(" ",1).str.get(0).str.strip()
Actions['Evénement']=Actions['Evénement'].str.split(" ",1).str.get(1).str.strip()
Actions['NrAction']=Actions['NrAction'].str.split(" ",1).str.get(0).str.strip()
Actions['SpkCommentaire']=Actions['RefCommentaire'].str.split("-",1).str.get(0).str.strip()
Actions['SpkCommentaire']=Actions['SpkCommentaire'].fillna("SansCommentaire")

###Calcul des références multiples de RefCommentaire

Certaines références contiennent plusieurs numéros de commentaire séparées par des virgules

In [12]:
refsTemp=list(Actions.RefCommentaire.values)
Refs=[]
for ref in refsTemp:
    if isinstance(ref, basestring):
        if "," in ref:
            temps=ref.split(",")
            for temp in temps:
                Refs.append(temp.strip())
        else:
            Refs.append(ref)

###Mise en forme de Commentaires

- Elimination des colonnes inutiles
- Calcul du RythmeArrondi
- Calcul de la référence du Commentaire et de son texte

In [13]:
Commentaires=Commentaires.drop('Evénement',1)
Commentaires=Commentaires.drop('NrAction',1)
Commentaires=Commentaires.drop('Action',1)
Commentaires=Commentaires.drop('ActionsAvant',1)
Commentaires=Commentaires.drop('ActionsAprès',1)
Commentaires["RythmeArrondi"]=np.round(Commentaires["RythmeAction"],0)
Commentaires['RefCommentaire']=Commentaires['Commentaire'].str.split(" ",1).str.get(0).str.strip()
Commentaires['Text']=Commentaires['Commentaire'].str.split(" ",1).str.get(1).str.strip()
#Commentaires['Longueur']=Commentaires['Text'].str.len()

###Calcul du type "action" vs "complémentaire" pour Commentaires['Type']

- si la référence du commentaire est dans la liste des références *Refs*
 - alors c'est un commentaire pour une action => "action"
 - sinon c'est du discours complémentaire => "complément"

In [14]:
def typeCommentaire(row):
    if row['RefCommentaire'] in Refs:
#        return "action-"+row['Speaker']
        return "action"
    else:
#        return "complémentaire-"+row['Speaker']
        return "complémentaire"
Commentaires['Type']=Commentaires.apply(typeCommentaire, axis=1)

###Rapprochement entre Syntaxe et Commentaires

- si pour la ligne courante *row*, le contenu *row['Text']* est dans la liste des commentaires annotés syntaxiquement
 - alors ajouter l'information à la colonne
 - sinon rien

In [15]:
def nbsyllabes(row):
    if row['Text'] in refsSyn:
        return Syntaxe.loc[Syntaxe["Commentaire"]==row['Text']]["NbSyllabes"].values[0]
    else:
        return ""
Commentaires['NbSyllabes']=Commentaires.apply(nbsyllabes, axis=1)
def nbmots(row):
    if row['Text'] in refsSyn:
        return Syntaxe.loc[Syntaxe["Commentaire"]==row['Text']]["NbMots"].values[0]
    else:
        return ""
Commentaires['NbMots']=Commentaires.apply(nbmots, axis=1)
def nbmots(row):
    if row['Text'] in refsSyn:
        return Syntaxe.loc[Syntaxe["Commentaire"]==row['Text']]["NbMots"].values[0]
    else:
        return ""
Commentaires['NbMots']=Commentaires.apply(nbmots, axis=1)
def debit(row):
    if row['Text'] in refsSyn:
        return Syntaxe.loc[Syntaxe["Commentaire"]==row['Text']]["Débit"].values[0]
    else:
        return ""
Commentaires['Débit']=Commentaires.apply(debit, axis=1)
def nbverbes(row):
    if row['Text'] in refsSyn:
        return Syntaxe.loc[Syntaxe["Commentaire"]==row['Text']]["NbVerbes"].values[0]
    else:
        return ""
Commentaires['NbVerbes']=Commentaires.apply(nbverbes, axis=1)
def nbnoms(row):
    if row['Text'] in refsSyn:
        return Syntaxe.loc[Syntaxe["Commentaire"]==row['Text']]["NbNoms"].values[0]
    else:
        return ""
Commentaires['NbNoms']=Commentaires.apply(nbnoms, axis=1)
def deco(row):
    if row['Text'] in refsSyn:
        return Syntaxe.loc[Syntaxe["Commentaire"]==row['Text']]["DeCo"].values[0]
    else:
        return ""
Commentaires['DeCo']=Commentaires.apply(deco, axis=1)

###Exportation des données pour les tableaux

####Tableaux généraux

In [16]:
Actions.to_csv("/Users/gilles/Downloads/Actions.csv", sep='\t', encoding='utf-8')
Commentaires.to_csv("/Users/gilles/Downloads/Commentaires.csv", sep='\t', encoding='utf-8')
Match.to_csv("/Users/gilles/Downloads/Match.csv", sep='\t', encoding='utf-8')

####Tableau Rythme

In [17]:
Rythme=Match[["Début","RythmeAction","RépartitionSpk1","RépartitionSpk2","RépartitionSpk3","RépartitionNoComm"]]
Rythme.to_csv("/Users/gilles/Downloads/TempsRythmeSpeaker.csv", sep='\t', encoding='utf-8')

####Tableau ActionsSpeakers

- extraction des trois plages de rythme
 - Rythme1 : RythmeAction <=rythme1
 - Rythme2 : rythme1 < RythmeAction <=rythme2
 - Rythme3 : rythme2 < RythmeAction

In [18]:
ActionsSpeakers=pd.DataFrame(columns=["Rythme1","Rythme2","Rythme3"])
ActionsSpeakers["Rythme1"]=Actions.loc[Actions["RythmeAction"]<=rythme1].groupby(["Action","SpkCommentaire"]).size()
ActionsSpeakers["Rythme2"]=Actions.loc[(Actions["RythmeAction"]>rythme1) & (Actions["RythmeAction"]<=rythme2)].groupby(["Action","SpkCommentaire"]).size()
ActionsSpeakers["Rythme3"]=Actions.loc[Actions["RythmeAction"]>rythme2].groupby(["Action","SpkCommentaire"]).size()
ActionsSpeakers.to_csv("/Users/gilles/Downloads/ActionsSpeakers.csv", sep='\t', encoding='utf-8')

####Tableau RythmesSpeakers

In [19]:
RythmesSpeakers=Commentaires.groupby(["RythmeArrondi","Speaker"]).size()
RythmesSpeakers.to_csv("/Users/gilles/Downloads/RythmesSpeakers.csv", sep='\t', encoding='utf-8')

####Tableau RythmesStructures

In [20]:
RythmesStructures=pd.DataFrame(columns=["Rythme1","Rythme2","Rythme3"])
RythmesStructures["Rythme1"]=Commentaires.loc[Commentaires["RythmeAction"]<=3].groupby(["Structure"]).size()
RythmesStructures["Rythme2"]=Commentaires.loc[(Commentaires["RythmeAction"]>3) & (Commentaires["RythmeAction"]<=6)].groupby(["Structure"]).size()
RythmesStructures["Rythme3"]=Commentaires.loc[Commentaires["RythmeAction"]>6].groupby(["Structure"]).size()
RythmesStructures.to_csv("/Users/gilles/Downloads/RythmesStructures.csv", sep='\t', encoding='utf-8')

####Tableau ActionsStructures

In [21]:
ActionsStructures=Match[["Action","RefCommentaire"]]
def structAction(row):
    temp=row["RefCommentaire"]
    commentaires=str(temp).split(",")
    structures=[]
    for commentaire in commentaires:
        if commentaire in Refs:
            valeur=str(Commentaires.loc[Commentaires["RefCommentaire"].str.lower()==commentaire]["Structure"].values[0]).strip()
            if valeur!="nan" and valeur!=" " and valeur!="":
                structures.append(valeur)
    return ";".join(structures)
ActionsStructures["Structures"]=ActionsStructures.apply(structAction, axis=1)
ActionsStructures["RefCommentaire"]=ActionsStructures["RefCommentaire"].fillna(value="SansCommentaire")
ActionsStructures.Structures[ActionsStructures.RefCommentaire=="SansCommentaire"]="SansCommentaire"
ActionsStructures.groupby(["Action","Structures"]).size().to_csv("/Users/gilles/Downloads/ActionsStructures.csv", sep='\t', encoding='utf-8')

-c:13: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/series.py:637: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  self.where(~key, value, inplace=True)


####Tableau StructuresSpeakers

In [22]:
StructuresSpeakers=Commentaires[["Structure","Speaker"]]
StructuresSpeakers.groupby(["Structure","Speaker"]).size().to_csv("/Users/gilles/Downloads/StructuresSpeakers.csv", sep='\t', encoding='utf-8')

####Tableau MotsDureeSpeaker

In [23]:
MotsDureeSpeaker=Commentaires[["Début","Fin"]]
MotsDureeSpeaker=pd.merge(MotsDureeSpeaker,Commentaires.loc[Commentaires["Speaker"].str.lower()=="spk1"][["Début","NbMots"]],on='Début',how='outer')
MotsDureeSpeaker.rename(columns={"NbMots":"Spk1"},inplace=True)
MotsDureeSpeaker=pd.merge(MotsDureeSpeaker,Commentaires.loc[Commentaires["Speaker"].str.lower()=="spk2"][["Début","NbMots"]],on='Début',how='outer')
MotsDureeSpeaker.rename(columns={"NbMots":"Spk2"},inplace=True)
MotsDureeSpeaker=pd.merge(MotsDureeSpeaker,Commentaires.loc[Commentaires["Speaker"].str.lower()=="spk3"][["Début","NbMots"]],on='Début',how='outer')
MotsDureeSpeaker.rename(columns={"NbMots":"Spk3"},inplace=True)
MotsDureeSpeaker.to_csv("/Users/gilles/Downloads/MotsDureeSpeaker.csv", sep='\t', encoding='utf-8')

####Tableau MotsDureeType

In [24]:
MotsDureeType=Commentaires[["Début","Fin"]]
MotsDureeType=pd.merge(MotsDureeType,Commentaires.loc[Commentaires["Type"].str.lower()=="action"][["Début","NbMots"]],on='Début',how='outer')
MotsDureeType.rename(columns={"NbMots":"Actions"},inplace=True)
MotsDureeType=pd.merge(MotsDureeType,Commentaires.loc[Commentaires["Type"].str.lower()=="complémentaire"][["Début","NbMots"]],on='Début',how='outer')
MotsDureeType.rename(columns={"NbMots":"Compléments"},inplace=True)
MotsDureeType.to_csv("/Users/gilles/Downloads/MotsDureeType.csv", sep='\t', encoding='utf-8')

#Zone d'élaboration : Run All Above

In [25]:
Commentaires.groupby(["Type","Structure"]).size().to_csv("/Users/gilles/Downloads/TypeStructures.csv", sep='\t', encoding='utf-8')

In [26]:
Commentaires.loc[Commentaires["Speaker"].str.lower()=="spk1"]["NbMots"]

2015-01-05 00:12:24.600000    
2015-01-05 00:12:28.560000    
2015-01-05 00:12:34.720000    
2015-01-05 00:12:36.960000    
2015-01-05 00:12:38.960000    
2015-01-05 00:12:40.880000    
2015-01-05 00:12:42.960000    
2015-01-05 00:12:45.400000    
2015-01-05 00:12:49.480000    
2015-01-05 00:12:50.320000    
2015-01-05 00:12:51.760000    
2015-01-05 00:12:54.760000    
2015-01-05 00:12:55.640000    
2015-01-05 00:12:57.520000    
2015-01-05 00:12:59.120000    
...
2015-01-05 01:49:58.060000    
2015-01-05 01:50:00.420000    
2015-01-05 01:50:02.580000    
2015-01-05 01:50:04.740000    
2015-01-05 01:50:05.980000    
2015-01-05 01:50:09.060000    
2015-01-05 01:50:10.260000    
2015-01-05 01:50:11.820000    
2015-01-05 01:50:13.220000    
2015-01-05 01:50:14.580000    
2015-01-05 01:50:16.420000    
2015-01-05 01:50:17.580000    
2015-01-05 01:50:18.340000    
2015-01-05 01:50:19.900000    
2015-01-05 01:50:20.900000    
Name: NbMots, Length: 1569

In [31]:
Commentaires.groupby(["Type","Structure1"]).size().to_csv("/Users/gilles/Downloads/TypeStructures1.csv", sep='\t', encoding='utf-8')
Commentaires.groupby(["Type","Structure2"]).size().to_csv("/Users/gilles/Downloads/TypeStructures2.csv", sep='\t', encoding='utf-8')
Commentaires.groupby(["Type","Structure3"]).size().to_csv("/Users/gilles/Downloads/TypeStructures3.csv", sep='\t', encoding='utf-8')

In [69]:
Comptage1=pd.DataFrame(Commentaires.groupby(["Type","Structure1"]).size()).unstack().fillna(0)
Comptage2=pd.DataFrame(Commentaires.groupby(["Type","Structure2"]).size()).unstack().fillna(0)
Comptage3=pd.DataFrame(Commentaires.groupby(["Type","Structure3"]).size()).unstack().fillna(0)

In [77]:
Comptage1.to_csv("/Users/gilles/Downloads/TypeStructures1.csv", sep='\t', encoding='utf-8')

In [78]:
Comptage2.to_csv("/Users/gilles/Downloads/TypeStructures2.csv", sep='\t', encoding='utf-8')

In [79]:
Comptage3.to_csv("/Users/gilles/Downloads/TypeStructures3.csv", sep='\t', encoding='utf-8')

In [ ]:
#RythmesStructures.to_csv("/Users/gilles/Downloads/ActionsSpeakers.csv", sep='\t', encoding='utf-8')

In [ ]:
#Match.loc[(Match['Action']!="") & (Match["RythmeAction"]>0)]['RythmeAction'].plot(figsize=(180,6))

Calcul du graphe de Longueur et Durée pour le spk1 en fonction du numéro de Sync (pas de la timeline)

In [ ]:
#Match.loc[(Match['Speaker']=="spk2")][["Durée","RythmeAction"]].plot(figsize=(180,6))

In [ ]:
#Match.loc[(Match['Action']!="") & (Match["Evénement"]=="3 porté")]["RythmeAction"].keys()[0]

In [ ]:
#Match.loc[(Match['Speaker']=="spk2")]

In [ ]:
#pd.rolling_mean(Match.loc[(Match['Action']!="") & (Match["RythmeAction"]>0)]['RythmeAction'],3).describe()

In [ ]:
#Extrait=Match.loc[(Match["Début"]>=Match["Début"][7]-datetime.timedelta(seconds=3))&(Match["Fin"]<=Match["Fin"][7]+datetime.timedelta(seconds=3))]
#Extrait.loc[Extrait["Speaker"].isnull()]["Speaker"].shape[0]


In [ ]:
#Commentaires["RythmeArrondi"]=np.round(Commentaires["RythmeAction"],0)

In [ ]:
#Commentaires.loc[Commentaires["RefCommentaire"]=="Spk2-071"]

In [ ]:
#for name, group in Actions.groupby(["Action","Speaker","RythmeAction"]):
#    print name[0], name[1], name[2], group.NrAction.count()

In [ ]:
#Match.loc[(Match['Action']!="") & (Match["RythmeAction"]>0)][["RythmeAction","RépartitionNoComm"]].plot(figsize=(180,6))

In [ ]:
#Commentaires